In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,058 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,776 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://

In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales_view")
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
Average_price4bd = spark.sql("SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS AveragePrice FROM home_sales_view WHERE bedrooms = 4 GROUP BY Year ORDER BY Year")
Average_price4bd.show()

+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
Average_price3bd = spark.sql("SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS AveragePrice FROM home_sales_view WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
Average_price3bd.show()


+---------+------------+
|YearBuilt|AveragePrice|
+---------+------------+
|     2010|   292859.62|
|     2011|   291117.47|
|     2012|   293683.19|
|     2013|   295962.27|
|     2014|   290852.27|
|     2015|    288770.3|
|     2016|   290555.07|
|     2017|   292676.79|
+---------+------------+



In [ ]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

Average_price3bdgreater2000sq = spark.sql("SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS AveragePrice FROM home_sales_view WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
Average_price3bdgreater2000sq.show()


+---------+------------+
|YearBuilt|AveragePrice|
+---------+------------+
|     2010|   285010.22|
|     2011|   276553.81|
|     2012|   307539.97|
|     2013|   303676.79|
|     2014|   298264.72|
|     2015|   297609.97|
|     2016|    293965.1|
|     2017|   280317.58|
+---------+------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

average_pricegreather350K = spark.sql("SELECT view, ROUND(AVG(price), 2) AS AveragePrice FROM home_sales_view WHERE price >= 350000 GROUP BY view")
average_pricegreather350K.show()

end_time = time.time()
execution_time = end_time - start_time

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|AveragePrice|
+----+------------+
|  31|   399856.95|
|  85|  1056336.74|
|  65|   736679.93|
|  53|    755214.8|
|  78|  1080649.37|
|  34|   401419.75|
|  81|  1053472.79|
|  28|   402124.62|
|  76|  1058802.78|
|  26|   401506.97|
|  27|   399537.66|
|  44|   400598.05|
|  12|   401501.32|
|  91|  1137372.73|
|  22|   402022.68|
|  93|  1026006.06|
|  47|    398447.5|
|   1|   401044.25|
|  52|   733780.26|
|  13|   398917.98|
+----+------------+
only showing top 20 rows

--- 0.833681583404541 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales_view")


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales_view")


True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result_cached = spark.sql("SELECT view, ROUND(AVG(price), 2) AS AveragePrice FROM home_sales_view WHERE price >= 350000 GROUP BY view")
result_cached.show()

end_time_cached = time.time()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------+
|view|AveragePrice|
+----+------------+
|  31|   399856.95|
|  85|  1056336.74|
|  65|   736679.93|
|  53|    755214.8|
|  78|  1080649.37|
|  34|   401419.75|
|  81|  1053472.79|
|  28|   402124.62|
|  76|  1058802.78|
|  26|   401506.97|
|  27|   399537.66|
|  44|   400598.05|
|  12|   401501.32|
|  91|  1137372.73|
|  22|   402022.68|
|  93|  1026006.06|
|  47|    398447.5|
|   1|   401044.25|
|  52|   733780.26|
|  13|   398917.98|
+----+------------+
only showing top 20 rows

--- 1.5176994800567627 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("path/to/partitioned_home_sales_data.parquet")

In [ ]:
# 11. Read the formatted parquet data.
parquet_data = spark.read.parquet("path/to/partitioned_home_sales_data.parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_data_view")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

filtered_df = df.filter(df.price >= 350000).groupBy("view").avg("price")
filtered_df = filtered_df.withColumn("average_price", round(filtered_df["avg(price)"], 2))
filtered_df.show()

runtime_parquet = time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+-------------+
|view|        avg(price)|average_price|
+----+------------------+-------------+
|  31|399856.95135135134|    399856.95|
|  85|1056336.7435897435|   1056336.74|
|  65| 736679.9333333333|    736679.93|
|  53|          755214.8|     755214.8|
|  78|1080649.3666666667|   1080649.37|
|  34| 401419.7451923077|    401419.75|
|  81|1053472.7878787878|   1053472.79|
|  28|402124.62176165805|    402124.62|
|  76|1058802.7777777778|   1058802.78|
|  26|401506.96774193546|    401506.97|
|  27| 399537.6586826347|    399537.66|
|  44|400598.04761904763|    400598.05|
|  12| 401501.3243243243|    401501.32|
|  91| 1137372.731707317|   1137372.73|
|  22| 402022.6847826087|    402022.68|
|  93|1026006.0606060605|   1026006.06|
|  47| 398447.4976744186|     398447.5|
|   1| 401044.2513368984|    401044.25|
|  52| 733780.2608695652|    733780.26|
|  13|         398917.98|    398917.98|
+----+------------------+-------------+
only showing top 20 rows

--- 0.76539587

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales_view")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales_view")


False